# The Best Neighborhood in Pittsburgh: Halloween Trick-or-Treating
## This neighborhood... 
* has the best sense of community
* great for families and children
* will provide the highest volume/best quality candy
* has high walkability rating